<img src="https://github.com/seap-udea/xearch/blob/main/resources/xearch-logo.png?raw=true" align="left">

## Required packages

In [1]:
# External modules
import spiceypy as spy
import numpy as np
import matplotlib.pyplot as plt
from astropy import constants as c
from astropy.time import Time
import pandas as pd
import pickle
from datetime import datetime
from tqdm import tqdm
from IPython.display import display,HTML

# Constants
G = c.G.value
Msun = c.M_sun.value
Mearth = c.M_earth.value
Mjup = c.M_jup.value
Rsun = c.R_sun.value
Rearth = c.R_earth.value
Rjup = c.R_jup.value
Mjup2earth = Mjup/Mearth
Rjup2earth = Rjup/Rearth

# Units
K = 1 # Kelvin
days = 86400
JD = days # Julian days
BJD = days # Bessel Julian days
years = 365.25*days # Julian years
au = c.au.value 
deg = np.pi/180
rad = 1/deg
Gyr = 1e9*years
dex = 1

# Useful routines
def sex2dec(angle):
    h, m, s = map(float, angle.split())
    sgn = np.sign(h)
    return sgn*(abs(h) + m/60 + s/3600)

def date2jd(date, format='%y/%m/%d'):
    dt = datetime.strptime(date, format)
    t = Time(dt)
    jd = t.jd
    return jd

def print_df(df):
    display(HTML(df.to_html()))


## Read database

In [211]:
# Read planetary systems
exec(open('signals/systems-full.py').read())

In [2]:
import xml.etree.ElementTree as ET, urllib.request, gzip, io
#url = "https://github.com/OpenExoplanetCatalogue/oec_gzip/raw/master/systems.xml.gz"
#url = "https://github.com/OpenExoplanetCatalogue/oec_gzip/raw/master/systems-exoplanetarchive.xml.gz"
url = "https://github.com/OpenExoplanetCatalogue/oec_gzip/raw/master/systems-exoplaneteu.xml.gz"

oec = ET.parse(gzip.GzipFile(fileobj=io.BytesIO(urllib.request.urlopen(url).read())))

In [3]:
# Output mass and radius of all planets 
for planet in oec.findall(".//planet"):
    name = planet.findtext("name") 
    if 'Proxima' in name:
        print(name)
        print(planet.findtext("mass"))
    pass

Proxima Centauri b
0.00337
Proxima Centauri d
0.00082


In [4]:
binary_properties = dict(
    # General
    name = ['binary_name',str,0],
    # Position
    separation = ['sep_bin',float,1,au], 
    positionangle = ['pa_bin',float,1,deg],
    # Orbital elements
    semimajoraxis = ['a_bin',float,1],
    eccentricity = ['e_bin',float,1],
    inclination = ['I_bin',float,1],
    periastron = ['w_bin',float,1,deg],
    ascendingnode = ['W_bin',float,1,deg],
    meananomaly = ['M_bin',float,1],
    meanlongitude = ['L_bin',float,1,deg],
    period = ['period_bin',float,1,days],
    epoch = ['et_bin',float,1,BJD],
    # Times
    maximumrvtime = ['tk_bin',float,1,BJD],
    # Magnitudes
    magB = ['magB_bin',float,1,dex],
    magV = ['magV_bin',float,1,dex],
    magI = ['magI_bin',float,1,dex],
    magR = ['magR_bin',float,1,dex],
    magJ = ['magJ_bin',float,1,dex],
    magH = ['magH_bin',float,1,dex],
    magK = ['magK_bin',float,1,dex],
) # Other: nstars

system_properties = dict(
    # General
    name = ['system_name',str,0],
    # Position
    declination = ['declination',sex2dec,1,deg],
    rightascension = ['rightascension',sex2dec,1,deg],
    distance = ['distance',float,1],
) # Other: nplanets

star_properties = dict(
    # General
    name = ['star_name',str,0],
    # Physical
    mass = ['stars_mass',float,1,Msun],
    radius = ['stars_radius',float,1,Rsun],   
    temperature = ['stars_temperature',float,1,K],
    age = ['stars_age',float,1,Gyr],
    # Spectral
    metallicty = ['metallicity',float,1,dex], # Relative to sun
    spectraltype = ['spectraltype',str,0],
    # Magnitudes
    magB = ['magB',float,1,dex],
    magV = ['magV',float,1,dex],
    magI = ['magI',float,1,dex],
    magR = ['magR',float,1,dex],
    magJ = ['magJ',float,1,dex],
    magH = ['magH',float,1,dex],
    magK = ['magK',float,1,dex],
)

planet_properties = dict(
    # General
    name = ['planet_name',str,0],
    # description = ['description',str,0],
    discoverymethod = ['method',str,0],
    # Times
    istransiting = ['istransiting',int,1],
    discoveryyear = ['dicovery',int,1,years],
    #lastupdate = ['update',date2jd,1,JD],
    lastupdate = ['update',str,0],
    # Physical    
    mass = ['mass',float,Mjup2earth,Mjup],
    radius = ['radius',float,Rjup2earth,Rjup],
    temperature = ['temperature',float,1,K],
    age = ['age',float,1,Gyr],
    # Position
    impactparameter = ['b',float,1],
    separation = ['separation',float,1,au],
    # Orbital elements
    semimajoraxis = ['a',float,1],
    eccentricity = ['e',float,1],
    inclination = ['I',float,1],
    periastron = ['w',float,1,deg],
    ascendingnode = ['W',float,1,deg],
    meananomaly = ['M',float,1],
    meanlongitude = ['L',float,1,deg],
    period = ['period',float,1,days],
    epoch = ['et',float,1,BJD],
    # Times
    transittime = ['tt',float,1,BJD],
    periastrontime = ['tp',float,1],
    maximumrvtime = ['tk',float,1,BJD],
    # Other
    spinorbitalignment = ['so',float,1,deg],
)
# Columns compilation
columns = []
for properties in system_properties,star_properties,planet_properties: #,binary_properties:
    for key,item in properties.items():
        columns.append(item[0])
print(f"Number of columns: {len(columns)}")

Number of columns: 42


Getting data:

In [6]:
nsystems = 0
nallplanets = 0 
systems = dict()

exoplanets_list = []
for system in tqdm(oec.findall('.//system')):
    name = system.findtext('name')
    if name is None:
        continue
    nsystems += 1
    
    systems[name] = dict()
    exoplanet = dict()

    system_dict = dict()
    # System properties
    for key,item in system_properties.items():
        value = system.findtext(key)
        if value is not None:
            finalvalue = item[1](value)
            if item[1] != str:
                finalvalue *= item[2]
        else:
            finalvalue = None
        systems[name][item[0]] = finalvalue
        system_dict[item[0]] = finalvalue

    # Stars
    nstars = 0
    systems[name]['stars'] = []
    for star in system.findall('.//star'):
        star_name = star.findtext('name') 
        if star_name is None:
            continue
        nstars += 1
        star_dict = dict()
        for key,item in star_properties.items():
            value = star.findtext(key)
            if value is not None:
                finalvalue = item[1](value)
                if item[1] != str:
                    finalvalue *= item[2]
            else:
                finalvalue = None

            star_dict[item[0]] = finalvalue
        systems[name]['stars'].append(star_dict)
        system_dict.update(star_dict)

    systems[name]['nstars'] = nstars
    
    # Planets
    nplanets = 0
    systems[name]['planets'] = []
    for planet in system.findall('.//planet'):
        planet_name = planet.findtext('name') 
        if planet_name is None:
            continue
        nallplanets += 1
        nplanets += 1
        planet_dict = dict()
        for key,item in planet_properties.items():
            value = planet.findtext(key)
            if (value is not None) and (value != ''):
                finalvalue = item[1](value)
                if item[1] != str:
                    finalvalue *= item[2]
            else:
                finalvalue = None

            planet_dict[item[0]] = finalvalue
            exoplanet[item[0]] = finalvalue

        systems[name]['planets'].append(planet_dict)
        system_dict_planet = system_dict.copy()
        system_dict_planet.update(planet_dict)

        exoplanets_list.append(system_dict_planet)

    systems[name]['nplanets'] = nplanets
    exoplanet['nplanets'] = nplanets

exoplanets = pd.DataFrame(exoplanets_list)
print_df(exoplanets.describe())

100%|██████████| 4279/4279 [00:00<00:00, 9313.75it/s]


,declination,rightascension,distance,stars_mass,stars_radius,stars_temperature,stars_age,magV,magI,magJ,magH,magK,istransiting,dicovery,mass,radius,temperature,a,e,I,w,W,period
count,5647.000000,5647.000000,5288.000000,5016.000000,4648.000000,4848.000000,9.00,2480.000000,192.000000,2938.000000,2926.000000,2240.000000,3928.0,5650.000000,3111.000000,4037.000000,1385.000000,3841.000000,2303.000000,1689.000000,1491.000000,9.000000,5.143000e+03
mean,16.997471,15.371422,705.068973,0.944882,1.556056,5426.485580,4.57,10.639099,12.939802,12.635192,12.255179,12.443361,1.0,2015.541593,1784.121906,4.921461,1034.060231,25.900614,0.168609,84.977731,163.217532,77.732910,2.679748e+03
std,36.863525,6.247880,1224.958446,0.478175,3.978377,1544.735669,0.00,3.250006,3.049992,1.578877,1.580841,1.508644,0.0,6.097612,3876.003293,5.555341,579.778037,325.410457,0.188840,15.440746,111.744316,42.291484,1.153930e+05
min,-88.121111,0.000000,1.295000,0.008000,0.008300,378.000000,4.57,0.850000,0.017000,2.850000,2.040000,2.170000,1.0,1781.000000,0.000667,0.010088,110.000000,0.002600,0.000000,-2.000000,-190.000000,-5.112604,1.960000e-02
25%,-12.783333,11.351389,84.536000,0.770000,0.777525,4943.750000,4.57,8.080000,11.955500,11.882000,11.494750,11.913000,1.0,2014.000000,15.891420,1.625302,580.000000,0.054000,0.025100,85.830000,72.935000,49.713210,4.287521e+00
50%,38.545581,18.948222,365.000000,0.946000,0.960000,5552.000000,4.57,10.700000,13.447000,12.939000,12.558000,12.706000,1.0,2016.000000,251.084441,2.488394,911.000000,0.118000,0.100000,88.240000,155.000000,76.672615,1.155103e+01
75%,45.197818,19.530694,830.250000,1.100000,1.250000,5909.000000,4.57,12.895000,14.743250,13.757000,13.338000,13.373500,1.0,2018.000000,1231.585076,5.738998,1425.000000,0.949000,0.250000,89.310000,260.000000,110.301680,4.715479e+01
max,85.736111,23.998333,11000.000000,20.000000,88.500000,42000.000000,4.57,25.000000,20.400000,19.600000,23.380000,16.310000,1.0,2024.000000,43224.663297,103.230229,3921.000000,9900.000000,0.970000,177.300000,360.000000,131.786359,8.035500e+06


In [7]:
def filter(dataframe, conditions=[]):
    cond = pd.Series([True]*len(dataframe))
    for condition in conditions:
        column = condition[0]
        value = condition[1]
        if isinstance(value,list):
            cond = (dataframe[column]>=value[0])&(dataframe[column]<=value[1])&cond
        else:
            cond = (dataframe[column] == value)&cond
    
    return dataframe[cond]

In [8]:
filter(exoplanets,[('system_name','ups And')])

,system_name,declination,rightascension,distance,star_name,stars_mass,stars_radius,stars_temperature,stars_age,metallicity,...,w,W,M,L,period,et,tt,tp,tk,so
5372,ups And,41.410556,1.613333,13.47,ups And,1.27,1.631,6212.0,NaN,None,...,44.519,NaN,None,None,4.61711,None,None,None,None,None
5373,ups And,41.410556,1.613333,13.47,ups And,1.27,1.631,6212.0,NaN,None,...,247.629,NaN,None,None,240.93700,None,None,None,None,None
5374,ups And,41.410556,1.613333,13.47,ups And,1.27,1.631,6212.0,NaN,None,...,252.991,NaN,None,None,1281.43900,None,None,None,None,None
5375,ups And,41.410556,1.613333,13.47,ups And,1.27,1.631,6212.0,NaN,None,...,7.300,NaN,None,None,3848.86000,None,None,None,None,None


In [9]:
print_df(filter(exoplanets,[('method','RV'),('e',[0.5,1])]).dropna(subset=['radius']).head(10))

,system_name,declination,rightascension,distance,star_name,stars_mass,stars_radius,stars_temperature,stars_age,metallicity,spectraltype,magB,magV,magI,magR,magJ,magH,magK,planet_name,method,istransiting,dicovery,update,mass,radius,temperature,age,b,separation,a,e,I,w,W,M,L,period,et,tt,tp,tk,so
754,HD 125390,38.966981,14.302589,154.3,HD 125390,1.36,6.17,4850.0,NaN,None,G7III,None,8.20,NaN,None,NaN,NaN,NaN,HD 125390 b,RV,NaN,2018.0,18/12/19,7043.077490,12.105699,NaN,None,None,None,1.5500,0.59100,NaN,342.400,NaN,None,None,1756.200000,None,None,None,None,None
772,HD 13167,-24.695411,2.137161,149.6,HD 13167,1.35,2.39,5671.0,NaN,None,G3V,None,8.34,NaN,None,NaN,NaN,NaN,HD 13167 b,RV,NaN,2018.0,18/12/20,1052.012026,13.080881,NaN,None,None,None,4.1000,0.56300,NaN,265.000,NaN,None,None,2613.000000,None,None,None,None,None
889,HD 163607,56.391961,17.894583,69.0,HD 163607,1.12,1.76,5522.0,NaN,None,G5IV,None,8.00,NaN,None,NaN,NaN,NaN,HD 163607 b,RV,NaN,2011.0,18/12/21,249.050339,14.224197,NaN,None,None,None,0.3620,0.74410,NaN,79.600,NaN,None,None,75.220300,None,None,None,None,None
1338,HD 80606,50.603611,9.376944,58.4,HD 80606,0.98,0.98,5645.0,NaN,None,G5,None,8.93,NaN,None,NaN,NaN,NaN,HD 80606 b,RV,1.0,2001.0,24/01/27,1323.469268,11.567668,NaN,None,None,None,0.4603,0.93183,89.24,301.213,NaN,None,None,111.436765,None,None,None,None,None
